In [1]:
#библиотеки, необходимые для формирования запросов сайту и получение ответов
#и поиска в полученной с сайта информации необхоимых для проекта данных
from bs4 import BeautifulSoup
import requests as req
import lxml.html as html
import urllib3

import time #библиотека для установки паузы между запросами к сайту

#общие библиотеки
import pandas as pd
import numpy as np

import re

In [2]:
#формируем header для включения его в запросы, так как сайты могут блокировать запросы без него
header = {'user-agent': 'Chrome/108.0.0.0'}
urllib3.disable_warnings()

In [5]:
url_star = 'https://www.booking.com/searchresults.en-us.html?label=gen173nr-1FCAEoggI46AdIM1gEaMIBiAEBmAExuAEXyAEM2AEB6AEB-AECiAIBqAIDuALlmsecBsACAdICJDFmMDhmM2QxLTMxYTAtNDMzMC05ODYwLTllNWU2M2EyMDZiN9gCBeACAQ&aid=304142&ss=Vienna%2C+Austria&ssne=Barcelona&ssne_untouched=Barcelona&efdco=1&lang=en-us&src=searchresults&dest_id=ChIJn8o2UZ4HbUcRRluiUYrlwv0&dest_type=city&place_id=ChIJn8o2UZ4HbUcRRluiUYrlwv0&latitude=48.2081743&longitude=16.3738189&ac_position=0&ac_click_type=g&ac_langcode=en-us&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=b46d5237308a0121&group_adults=2&no_rooms=1&group_children=0&sb_travel_purpose=leisure&nflt=ht_id%3D204%3Bclass%3D'
url_all_star = []

for i in range(2):
    url_all_star.append(url_star + str(i+4) + '&order=distance_from_search')

In [9]:
urls_hotel_4 = []
urls_hotel_5 = []

for star in range(6):
    time.sleep(3)  
    if star == 0:
        
        for i in range(0, 201, 25):
            try:
                response = req.get(url_all_star[star] + '&offset=' + str(i), headers=header, timeout=50, verify=False)
                page = BeautifulSoup(response.text,'html.parser')
            except:
                print('Ошибка запроса')
                
                
            for a in page.find_all('a', class_='e13098a59f'):
                urls_hotel_4.append(a['href'])
        time.sleep(3) 
    if star == 1:
        for i in range(0, 26, 25):
            try:
                response = req.get(url_all_star[star] + '&offset=' + str(i), headers=header, timeout=50, verify=False)
                page = BeautifulSoup(response.text,'html.parser')
            except:
                print('Ошибка запроса')
                
                
            for a in page.find_all('a', class_='e13098a59f'):
                urls_hotel_5.append(a['href'])
        time.sleep(3)
    


print(len(urls_hotel_4))
print(len(urls_hotel_5))

221
28


In [10]:
urls_hotel_4_en = []
urls_hotel_5_en = []

    
for i in range(len(urls_hotel_4)):
    parts = urls_hotel_4[i].split('hapos=' + str(i+1))
    urls_hotel_4_en.append(parts[0] + ('hapos=' + str(i+1)) + '&lang=en-us' + parts[1])
    
for i in range(len(urls_hotel_5)):
    parts = urls_hotel_5[i].split('hapos=' + str(i+1))
    urls_hotel_5_en.append(parts[0] + ('hapos=' + str(i+1)) + '&lang=en-us' + parts[1])


print(len(urls_hotel_4_en))
print(len(urls_hotel_5_en))

221
28


In [11]:
data_array_full = np.empty(4, 'str')

tags = ['span', 'div', 'div']
classes = ['hp_address_subtitle js-hp_address_subtitle jq_tooltip', 'd8eab2cf7f c90c0a70d3 db63693c62',
           'b5cd09854e f0d4d6a2f5 e46e88563a']

all_star_url = [urls_hotel_4_en, urls_hotel_5_en]


count_star_0 = 3

for urls in all_star_url:
    count_star_0 += 1
    for url in urls:
        try:
            response = req.get(url, headers=header, timeout=50, verify=False)
            page = BeautifulSoup(response.text,'html.parser')
        except:
            print('ошибка запроса')
        
    
        data = []
        data.append(count_star_0)
        for i in range(3):
            try:
                elem = page.find_all(tags[i], classes[i])
                elem = elem[0].text
            except:
                elem = np.NaN
        
                
            data.append(elem)
            
        data_array = np.asarray(data).reshape(1,4)
        data_array_full = np.row_stack((data_array_full,data_array))
    
        time.sleep(3)

ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса


In [50]:
columns = np.array(['hotel_star', 'hotel_address','reviews_count','quality'])
hotel_vienna = pd.DataFrame(data=data_array_full, columns=columns)
hotel_vienna.replace('nan', np.NaN, inplace=True)
hotel_vienna = hotel_vienna.dropna(thresh=2).reset_index(drop=True)
hotel_vienna

,hotel_star,hotel_address,reviews_count,quality
0,,,,
1,4,"\nSingerstraße 3, 01. Innere Stadt, 1010 Vienn...","2,207 reviews",Very Good
2,4,"\nSchulerstraße 10, 01. Innere Stadt, 1010 Vie...",883 reviews,Wonderful
3,4,"\nSchulerstraße 14, 01. Innere Stadt, 1010 Vie...","3,310 reviews",Excellent
4,4,"\nRotenturmstraße 1-3 7. Stock, Top 22, 01. In...",213 reviews,Exceptional
...,...,...,...,...
245,5,"\nLeopoldsgasse 39, 02. Leopoldstadt, 1020 Vie...",58 reviews,Exceptional
246,5,"\nBurggasse 2, 07. Neubau, 1070 Vienna, Austria\n",925 reviews,Wonderful
247,5,"\nNordwestbahnstrasse 37A, 20. Brigittenau, 12...","2,044 reviews",Wonderful
248,5,"\nArsenalstrasse 10, 1100 Vienna, Austria\n","1,453 reviews",Wonderful


In [51]:
hotel_df = pd.read_csv('C:/Users/Dmitriy/Desktop/SkillFactory/project_3/hotels.csv', sep=',')
hotel_df.shape

(386803, 57)

In [54]:
hotel_vienna['hotel_address'] = hotel_vienna['hotel_address'].apply(
    lambda x: re.sub("[^0-9a-zA-Z]"," ",x).strip().lower().replace(' ', ''))

C:\Users\Dmitriy\AppData\Local\Temp\ipykernel_16464\1452831438.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_vienna['hotel_address'] = hotel_vienna['hotel_address'].apply(


In [55]:
hotel_vienna['hotel_address']

0                                                       
1             singerstrae301innerestadt1010viennaaustria
2           schulerstrae1001innerestadt1010viennaaustria
3           schulerstrae1401innerestadt1010viennaaustria
4      rotenturmstrae137stocktop2201innerestadt1010vi...
                             ...                        
245       leopoldsgasse3902leopoldstadt1020viennaaustria
246                  burggasse207neubau1070viennaaustria
247    nordwestbahnstrasse37a20brigittenau1200viennaa...
248                    arsenalstrasse101100viennaaustria
249         donaucitystrae722donaustadt1220viennaaustria
Name: hotel_address, Length: 239, dtype: object

In [56]:
hotel_df['hotel_address'] = hotel_df['hotel_address'].apply(
    lambda x: x.lower().replace(' ', ''))

In [57]:
hotel_df['hotel_address']

0         strattonstreetmayfairwestminsterboroughlondonw...
1         130134southamptonrowcamdenlondonwc1b5afunitedk...
2                   151bisruederennes6tharr75006parisfrance
3                216avenuejeanjaures19tharr75019parisfrance
4                      molenwerf11014agamsterdamnetherlands
                                ...                        
386798    9knaresboroughplacekensingtonandchelsealondons...
386799    landstraerhauptstrae15503landstrae1030viennaau...
386800      2931gowerstreetcamdenlondonwc1e6hgunitedkingdom
386801    31greatcumberlandplacewestminsterboroughlondon...
386802    25courtfieldgardenskensingtonandchelsealondons...
Name: hotel_address, Length: 386803, dtype: object

In [58]:
hotel_df = hotel_df.merge(
                hotel_vienna,
                on='hotel_address',
                how='left')

In [59]:
null = hotel_df[hotel_df['city_hotel'] == 'Vienna'].isnull().mean() * 100
null_per = null[null > 0]
null_per

hotel_star       16.892864
reviews_count    17.417232
quality          17.417232
dtype: float64

In [52]:
dupl_columns = list(hotel_vienna.columns) 

mask = hotel_vienna.duplicated(subset=dupl_columns) 
hotel_duplicates = hotel_vienna[mask]
print(f'Число найденных дубликатов: {hotel_duplicates.shape[0]}')

Число найденных дубликатов: 11


In [53]:
hotel_vienna = hotel_vienna.drop_duplicates(subset=dupl_columns)
print(f'Результирующее число записей: {hotel_vienna.shape[0]}')

Результирующее число записей: 239


In [60]:
hotel_vienna.to_csv('C:/Users/Dmitriy/Desktop/SkillFactory/project_3/parsing_hotels/hotel_vienna.csv')